In [1]:
ref='nb12-' 

import pandas as pd
import numpy as np

%matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatch
import matplotlib
plt.style.use('seaborn-whitegrid')
matplotlib.rcParams.update({
    'font.family': 'serif',
    'axes.titlesize':16,
    'axes.labelsize':16,
    'axes.xmargin':0.1,
    'axes.ymargin':0.1,
    'legend.fontsize':16,
    'xtick.labelsize' : 16,
    'ytick.labelsize' : 16,
    'lines.markersize': 10,
    'lines.linewidth' : 2,
    'font.size': 16
})


import datetime
import copy
import os


Using matplotlib backend: Qt5Agg


In [2]:
data=pd.read_csv("../data/processed/nb11-annot_gathered.csv")

In [3]:
#Input SubjectId and RunID
selected_subject=52.0 #shall be float
selected_run=1.0 #shall be float
skip=0 #0,1,2,3..etc # skipped samples (i.e. downsampling)
n_subplots=3 #no. of subplots 1,2,3,..etc

In [4]:
# Fetch walk
walk_data=data[data['SubjectId']==selected_subject]
walk_data=walk_data[walk_data['RunId']==selected_run]
walk_data.reset_index(drop=True,inplace=True)
if skip != 0:
    walk_data=walk_data.iloc[::skip]
    walk_data.reset_index(drop=True,inplace=True)
#subdata=np.array_split(data.values,n_subplots)


In [5]:
#Get min and max accls in walk
min_accl=np.min(walk_data.values[:,3:6])
max_accl=np.max(walk_data.values[:,3:6])
#Fetch segments per walk
segments=walk_data['Annotation']!=walk_data['Annotation'].shift()
segments=walk_data[segments]
seg_idx=segments.index
segment_data=[walk_data.iloc[seg_idx[i]:seg_idx[i+1]] for i in range(len(seg_idx)-1)]
segment_data.append(walk_data.iloc[seg_idx[-1]:]) 
annot=segments['Annotation'].values
annot_type=segments['AnnotationType'].values
segment_names = [i + ' - ' + j for i, j in zip(annot, annot_type)]
#segment_names=segments['Annotation'].values
segment_widths=[len(seg) for seg in segment_data]

In [6]:
# SMK walk_data[walk_data['speed']==np.inf].index

In [7]:

walk_data=walk_data.values
segment_rects=[]
for i in range(len(segment_names)): #Find rectangle borders (in x-axis) for each specific annotation
    # rect_y1=np.min(walk_data[:,3])
    # rect_y2=np.max(walk_data[:,3])
    rect_y1=min_accl
    rect_y2=max_accl
    segment_rects.append(mpatch.Rectangle((segment_data[i].index[0],rect_y1), segment_widths[i],rect_y2-rect_y1, fill=False))


plt.rcParams["figure.figsize"] = (20,14)
fig,ax=plt.subplots()
acc_x, =ax.plot(walk_data[:,3], label='x') #plot x (vertical)
acc_y, =ax.plot(walk_data[:,4], label='y') #plot y
acc_z, =ax.plot(walk_data[:,5], label='z') #plot z
for i,rect in enumerate(segment_rects):
    ax.add_artist(rect)
    rx, ry = rect.get_xy()
    cx = rx + rect.get_width()/2.0
    cy = ry + rect.get_height()
    ax.annotate(segment_names[i], (cx, cy), color='k', size=10, ha='center', va='center', rotation='vertical')

ax.set_title('Progression in time for Subject {} and Run {} downsampled by {}'.format(selected_subject,selected_run,skip), pad=25)
ax.set_xlabel('Time steps')
ax.set_ylabel('Acceleration')

ax2=ax.twinx()
speed, =ax2.plot(walk_data[:,6],color='y', label='speed') #plot speed

ax.legend(handles=[acc_x,acc_y,acc_z, speed])
plt.show()



NVD3D10: CPU cyclestats are disabled on client virtualization 
NVD3D10: CPU cyclestats are disabled on client virtualization 


In [8]:

plt.savefig("../reports/figures/{}-annot_{}_{}_{}.png".format(ref,selected_subject,selected_run,skip))